<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Парсинг" data-toc-modified-id="Парсинг-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Парсинг</a></span></li><li><span><a href="#Очистка" data-toc-modified-id="Очистка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Очистка</a></span></li></ul></div>

Здесь реализован синхронный парсер картин с [сайта](https://gallerix.ru/storeroom/) галереи. На страничке есть поблочный список авторов у которых более 100 картин. У каждого автора есть свой альбом, в котором можно последовательно двигаться по работам. Бот сохраняет картины в папке в читаемом виде для dataloader из pytorch.

## Парсинг

In [1]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import notebook
import re
from PIL import Image
import shutil

In [2]:
headers = {
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.174 YaBrowser/22.1.2.834 Yowser/2.5 Safari/537.36'
}

In [3]:
def get_soup(url, headers):
    req = requests.get(url, headers=headers)
    src = req.text
    soup = BeautifulSoup(src, 'lxml')
    return soup

In [4]:
url = 'https://gallerix.ru/storeroom/'
soup = get_soup(url, headers)

In [5]:
blocks = soup.find_all(class_='sr-pntrs')
auths = []
for block in blocks:
    auths.extend(block.find_all('a'))
len(auths)

240

In [6]:
os.mkdir('dataset')

#Проход по авторам
for auth in notebook.tqdm(auths):
    path = auth.text
    if 'REDIRECT' in path:
        continue
    os.mkdir(f'dataset/{path}')
    url = 'https://gallerix.ru' + auth.get('href')

    #Спуск к альбому
    try:
        soup = get_soup(url, headers)
        url = 'https://gallerix.ru' + soup.find(class_='pic').find(class_='animsition-link').get('href')
    except:
        print('cant go to the auth page: ', path)
        continue
        
    #Сбор картин
    img_num = 0
    while True:
        try:
            soup = get_soup(url, headers)
            name = re.sub(r'[^\w\s]','', soup.find(class_='panel-title').text)
            img_src = soup.find(class_='xpic').find('img').get('src')
            with open(f'dataset/{path}/{img_num}.jpg', 'wb') as img:
                img.write(requests.get(img_src, headers=headers).content)
            img_num += 1
        except:
            print('cant save image: ', path, name)
        try:
            fol = soup.find(class_='icon ion-md-arrow-forward d2 m2').find_parent()
            if fol.get('onclick') == 'return:false;':
                break
            url = 'https:' + fol.get('onclick')[15:-2]
        except:
            print('cant go on to the next link in: ', path, name)
            break

  0%|          | 0/240 [00:00<?, ?it/s]

cant go to the auth page:  Burliuk, David Davidovich
cant save image:  Challenger, J. D. Страница не найдена
cant go on to the next link in:  Challenger, J. D. Страница не найдена
cant go to the auth page:  Chirico, Giorgio De
cant go to the auth page:  Dali, Salvador
cant go to the auth page:  Deyneka, Alexander
cant go to the auth page:  Dix, Otto
cant save image:  Driben, Peter drib 047 Питер Дрибен
cant save image:  Earle, Eyvind Страница не найдена
cant go on to the next link in:  Earle, Eyvind Страница не найдена
cant save image:  Elvgren, Gil GIL012 Джил Элвгрен 19141980
cant save image:  Elvgren, Gil p Avalanche Pinup Cal1995 04 Джил Элвгрен 19141980
cant go to the auth page:  Ernst, Max
cant save image:  Fisher, Harrison Страница не найдена
cant go on to the next link in:  Fisher, Harrison Страница не найдена
cant save image:  Lebourg, Albert-Charles Береговая линия порта Онфлёра АльберШарль Лебур
cant go to the auth page:  Lempicka, Tamara De
cant go to the auth page:  Magrit

In [7]:
root = 'dataset'
num_auth = 0
num_img = 0

# проходимся по всем подпапкам и файлам внутри директории root
for dirname, _, files in os.walk(root):
    if dirname == 'dataset':
        continue
    num_auth += 1
    num_img += len(files)
print('num_auth: ', num_auth)
print('num_img: ', num_img)

num_auth:  239
num_img:  46123


Не все картины удалось собрать. Теперь необходимо проверить целостность изображений и удалить авторов у которых работ получилось меньше 100 

## Очистка

In [8]:
num_del = 0
for dirname, _, files in notebook.tqdm(os.walk(root)):
    if dirname == 'dataset':
        continue
    for img in files:
        try:
            Image.open(f'{dirname}/{img}')
        except:
            os.remove(f'{dirname}/{img}')
            num_del += 1
print('num_del: ', num_del)

0it [00:00, ?it/s]

num_del:  0


In [9]:
root = 'dataset'
num_auth = 0
num_img = 0

# проходимся по всем подпапкам и файлам внутри директории root
for dirname, _, files in os.walk(root):
    if dirname == 'dataset':
        continue
    if len(files) < 100:
        shutil.rmtree(dirname)
        continue
    num_auth += 1
    num_img += len(files)
print('num_auth: ', num_auth)
print('num_img: ', num_img)

num_auth:  218
num_img:  45747


Итого 218 авторов и 45747 работ